In [1]:
from fit import *
import matplotlib.pyplot as plt
%matplotlib widget
from utils import plot_event, plot_generator, get_data_files, preceding_noise_files
from counter_config import CounterConfig, init_config

In [2]:
data_date_and_time = '20200818040605'
cfg = init_config(data_date_and_time)

In [5]:
len(cfg.active_counters)

11

In [4]:
corsika_file = '/home/isaac/NIZ/he_corsika/iact_DAT000004.dat'
pe = ProcessEvents(cfg, frozen_noise=True)
real_nfits = pe.gen_nfits_from_ei(corsika_file)

ValueError: operands could not be broadcast together with shapes (11,) (51200,12) 

In [ ]:
real_nfits

In [ ]:
ty = tyro(real_nfits)
pf = NichePlane(real_nfits)
plot_event(ty)

In [ ]:
import CHASM as ch
ei = ch.EventioWrapper(corsika_file)
print(ei.X[ei.nch.argmax()])
print(ei.nch.max())
print(ei.theta)

In [ ]:
guess = make_guess(ty, pf,cfg)
[print(p) for p in guess];

In [ ]:
guess = BasicParams.adjust_guess(guess)

In [ ]:
pt = PeakTimes(real_nfits, BasicParams, cfg)
pt.target_parameters = ['zenith','azimuth']
m = init_minuit(pt, guess)
m.simplex(ncall=40)

In [ ]:
tpguess = update_guess(m)
pw = PulseWidth(real_nfits, BasicParams, cfg)
pw.target_parameters = ['xmax']
m = init_minuit(pw, tpguess)
m.simplex(ncall=40)

In [ ]:
xmaxguess = update_guess(m)
pa = PulseArea(real_nfits, BasicParams, cfg)
pa.target_parameters = ['nmax']
m = init_minuit(pa, xmaxguess)
m.simplex(ncall=20)

In [ ]:
nmaxguess = update_guess(m)
pa = NormalizedPulseArea(real_nfits, BasicParams, cfg)
pa.target_parameters = ['xmax','nmax','corex','corey']
m = init_minuit(pa, nmaxguess)
m.simplex()

In [ ]:
guess = update_guess(m)
at = AllTunka(real_nfits, BasicParams, cfg)
at.target_parameters = ['t_offset']
m = init_minuit(at, guess)
m.migrad()

In [ ]:
guess = update_guess(m)
at = AllSamples(real_nfits, BasicParams, cfg)
at.target_parameters = ['t_offset']
m = init_minuit(at, guess)


In [ ]:
m.tol=.01
m.fixed = True
m.fixed['xmax'] = False
m.fixed['nmax'] = False
m.fixed['zenith'] = False
m.fixed['azimuth'] = False
m.fixed['corex'] = False
m.fixed['corey'] = False
# m.fixed['t_offset'] = False
# m.scan()
m.simplex()

In [ ]:
m.tol=.01
m.fixed = True
m.fixed['xmax'] = False
m.fixed['nmax'] = False
m.fixed['lambda'] = False
# m.scan()
m.simplex()

In [ ]:
import CHASM as ch
ei = ch.EventioWrapper(corsika_file)
print(ei.X[ei.nch.argmax()])
print(ei.nch.max())
print(ei.theta)

In [ ]:
fitpars = [p.value for p in m.params]
fit_ev = BasicParams.get_event(fitpars)
print(fit_ev)

In [ ]:
fit_nfits = pe.gen_nfits_from_event(fit_ev)
plot_event(tyro(fit_nfits))

In [ ]:
plt.figure()
fitpars = [p.value for p in m.params]
plt.errorbar(np.arange(len(at.real_values)),at.real_values,yerr=at.error, fmt="o")
plt.plot(at.get_output(fitpars))

In [ ]:
at.chi2(fitpars)/m.ndof

In [ ]:
at.get_output(fitpars)

In [ ]:
for f in at.nfits:
    plt.figure()
    plt.plot(f.waveform)
    plt.hlines(f.baseline + f.baseline_error,0,1040, color='k')
    plt.hlines(f.baseline,0,1040, color='r')
    plt.vlines(f.start_rise, f.baseline,f.waveform.max())
    plt.xlim(f.start_rise,f.end_fall)


In [ ]:
cfg.data_files